# Sprint24

In [10]:
'''Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
# Summary of the algorithm
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
# Data download
English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/
# References
- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

# 学習パラメータ設定
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

# Vectorize the data.
input_texts = []  # インプットテキスト
target_texts = [] # ターゲットテキスト
input_characters = set()  # インプット文字
target_characters = set() # ターゲット文字
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n') # 改行コードで区切る
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t') # 「tab」で区切る
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    # 出現文字を一文字づつ保存
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# 文字を降順に並び替え
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# 文字の数
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# 文章の最大長さ
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# 文字を数字に変換
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# エンコード、デコード用の変数を用意
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# 文字ごとにOne Hot Encoding
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# 入力処理
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 隠れ層の状態、h, c を保存
encoder_states = [state_h, state_c]

# 初期は `encoder_states`を使ってデコーダを設定
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# デコーダは、出力、内部状態も返すように設定。 
# 戻り値の状態はトレーニングモデルでは使用しない。
# 推論で使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# `encoder_input_data`と` decoder_input_data`を `decoder_target_data`に変換するモデルを定義
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next：推論モード（サンプリング）
# 1）入力をエンコードして初期デコーダ状態を取得
# 2）この初期状態と「開始シーケンス」トークンを
# 目標として１ステップのデコーダを実行
# 3）現在のターゲットトークンと現在の状態で繰り返す

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

# 隠れ層の状態を取得する変数
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# デコーダを取得した状態で初期化
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

# モデル作成
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# シーケンスを読み取り可能なものにデコードするための
# 逆引きトークンインデックス
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.9188 - val_loss: 0.9395
Epoch 2/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.7297 - val_loss: 0.7783
Epoch 3/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.6209 - val_loss: 0.6956
Epoch 4/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.5655 - val_loss: 0.6461
Epoch 5/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.5259 - val_loss: 0.6002
Epoch 6/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.4935 - val_loss: 0.5798
Epoch 7/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.4671 - val_loss: 0.5581
Epoch 8/100
8000/8000 [==============================] - 15

8000/8000 [==============================] - 15s 2ms/step - loss: 0.0803 - val_loss: 0.6754
Epoch 78/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.0790 - val_loss: 0.6753
Epoch 79/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.0782 - val_loss: 0.6818
Epoch 80/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.0769 - val_loss: 0.6843
Epoch 81/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.0755 - val_loss: 0.6956
Epoch 82/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.0746 - val_loss: 0.7013
Epoch 83/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.0733 - val_loss: 0.6990
Epoch 84/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.0727 - val_loss: 0.6989
Epoch 85/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.0712 - val_loss: 0.7182
Epoch 86/100
8000/8000 [==============================] - 14s 2ms/st

C:\Users\G2\Anaconda3\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_10 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_9/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_9/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


-
Input sentence: Go.
-
Input sentence: Hi.
-
Input sentence: Run!
-
Input sentence: Run!
-
Input sentence: Wow!
-
Input sentence: Fire!
-
Input sentence: Help!
-
Input sentence: Jump.
-
Input sentence: Stop!
-
Input sentence: Stop!
-
Input sentence: Stop!
-
Input sentence: Wait!
-
Input sentence: Wait!
-
Input sentence: Go on.
-
Input sentence: Go on.
-
Input sentence: Go on.
-
Input sentence: Hello!
-
Input sentence: Hello!
-
Input sentence: I see.
-
Input sentence: I try.
-
Input sentence: I won!
-
Input sentence: I won!
-
Input sentence: Oh no!
-
Input sentence: Attack!
-
Input sentence: Attack!
-
Input sentence: Cheers!
-
Input sentence: Cheers!
-
Input sentence: Cheers!
-
Input sentence: Cheers!
-
Input sentence: Get up.
-
Input sentence: Go now.
-
Input sentence: Go now.
-
Input sentence: Go now.
-
Input sentence: Got it!
-
Input sentence: Got it!
-
Input sentence: Got it?
-
Input sentence: Got it?
-
Input sentence: Got it?
-
Input sentence: Hop in.
-
Input sentence: Hop in.
-
I

# Kerasで動かしたい場合はどうするか
1. Pytorch を Keras に書き換える
2. Pytoch の重みを hdf5 形式で保存する <br>
torch.save(model_trained.state_dict(),'models/pytorch/weights.h5')

# 説明

# 今回使用したSeq2Seqの実装を使い日本語と英語の翻訳を行いたい場合はどのような手順を踏むか
1. 日本語を mecab 等を使用して形態素解析し、分かち書きする
2. 日本語と英語をベクトル化する
3. 日本語と英語をエンコーダ、デコーダに通す

# 機械翻訳のための発展的手法にはどういったものがあるか

1. ルール型機械翻訳　【Rule-Based Machine Translation (RMT)】<br>
    文法規則に比重を置いた機械翻訳。単語やフレーズや文法ごとに、訳出ルールをコンピューターに登録し、そのルールに従って翻訳していく仕組み。基本的には、単語や文法などの意味を一つ一つ登録していく作業が必要とされる。<br>
    難点は、ルールに当てはまらない表現は正確な訳出が出来ません。<br>
    

2. 統計型機械翻訳　【Statistical Machine Translation (SMT)】<br>
    辞書／用語集（コーパス）を解析し、統計、確率を基にした機械翻訳。これは一定ルールに固定されるのではなく、コーパスとして収集した大量の対訳データの中から統計/確率結果として最も適した訳出を割り出していく仕組み。この手法の方が、ルール型翻訳に比して、より人間の言語処理能力に近い形であるのが特徴。つまりちょうど統計型翻訳システムが膨大な対訳データから適訳を割り出していく仕組みのように、人間もある一つの固定されたルールではなく、豊富な経験から状況に応じて言語処理をしているからである。この仕組みによって必然的に、精度も高まった。<br>
    難点としては、まず第一に、コーパスの充実、要するに大量の対訳データが精度向上のためには必須であるということ (データにない訳出はできないため)、もう一つは、その対訳データの解析力の幅にも限界があるということ。言い換えれば、対訳データとして一度に正確に処理できる言語量あるいは領域が狭いので、例えば、単文ではなく、復文や複雑で長い文構造になると語と語のつながり等、解析力の範囲を超えてしまい、正確な訳出が不能になってしまう。<br>


3. ニューラルネットワーク型機械翻訳　【Neural Machine Translation (NMT)】<br>
    多層的「ニューラルネットワーク」いわゆる 「ディープラーニング」と呼ばれる人口知能(AI)に自動学習させる翻訳です。ディープラーニングの考えのもととなっているニューラルネットワークは、人間の脳神経回路を真似することによってデータを分類しようというアイデアに基づくアルゴリズムのこと。この点で、統計型翻訳の仕組みをさらに進化させたものであり、例えば、統計型では語句単位での対訳データ解析とも言えますが(文の各部分をひとつひとつ訳出していく)、ニューラルネットワーク型では、復文も含めた文全体単位でのより多層的な解析を実現している。(翻訳システムは、まず文全体の構造や形を認識し、それによって文のコンテキストおよび各部位を理解し訳出していく) そして最後に、出来上がったセンテンスを正しい文法に則って調整し並べ替えることも行う。これによって既存の機械翻訳と比較し、翻訳エラーが55～85％減少したとも検証されている上に、より流暢な翻訳が可能となり、人間による翻訳に近づいた、より自然な翻訳結果が表示されるようになっている。しかもニューラルネットワークシステムの特性として、時間の経過とともに自分を改良し、長く使えば使うほどより自然な翻訳ができるようになる。 

# イメージキャプショニングとは逆に文章から画像を生成する手法にはどういったものがあるか
Microsoft Researchで開発中の新しいAIテクノロジは、キャプションのような文章中の単語に注目して画像を生成する。[arXiv.org](https://arxiv.org/abs/1711.10485)に投稿した研究論文のテストの結果では、このテクノロジは、従来のテキストから画像への変換手法の最先端技術と比較して、業界標準テストによる画像品質が約3倍に向上している。

研究者が「ドローイングボット」と呼ぶこのテクノロジは、家畜の群れる農村の風景などの普通の景色から、宙に浮く2階建てバスといった馬鹿げたものまで、テキストからあらゆる画像を生成できる。各画像には、テキストの説明にはない詳細な背景などが含まれ、このAIには人工的な想像力を有していることを物語っている。

# 異なるフレームワークに実装を移す場合にはどういった部分を気をつけるか
1. フレームワークごとに書き方、APIが異なるので、各フレームワークにあった書き方をする
2. フレームワークにより、保存するモデルや重みにファイル形式が異なるので、そのまま流用できるか確認をする
3. 正常に動いたとしても正しい結果になっているとは限らないので、必ず移植前と後でデータを確認する